In [1]:
import pandas as pd
import pm4py
from datetime import datetime
import numpy as np
from tqdm import tqdm 

In [2]:
application_log = pm4py.read_xes('BPI Challenge 2017.xes')
offer_log = pm4py.read_xes('BPI Challenge 2017 - Offer log.xes')

df_application = pm4py.convert_to_dataframe(application_log)
df_offer = pm4py.convert_to_dataframe(offer_log)

df_application.to_csv('app_logs.csv')
df_offer.to_csv('offer_logs.csv')

parsing log, completed traces :: 100%|██████████| 42995/42995 [00:10<00:00, 4021.40it/s]


In [4]:
df_application = pd.read_csv('./../../Assignment_2/Data/app_logs.csv')
df_offer = pd.read_csv('./../../Assignment_2/Data/offer_logs.csv')

In [46]:
declined_ids = list(df_application.loc[df_application['concept:name'] == 'A_Denied']['case:concept:name'].unique())
accepted_ids = list(df_application.loc[df_application['concept:name'] == 'A_Pending']['case:concept:name'].unique())
cancelled_ids = list(df_application.loc[df_application['concept:name'] == 'A_Cancelled']['case:concept:name'].unique())

In [47]:
# 31,509 loan applications in total
# 98 noise 
len(declined_ids)+ len(accepted_ids)+len(cancelled_ids)

31411

In [48]:
len(declined_ids), len(accepted_ids), len(cancelled_ids)

(3752, 17228, 10431)

In [49]:
def get_min_max_time(ids, i):
    timestamps = df_application.loc[df_application['case:concept:name'] == ids[i]]['time:timestamp']
    return (min(timestamps), max(timestamps))

In [50]:
# declined_ids: 0.19175868569889576
# cancelled_ids: 0.19131614654002713
# approved_ids: 0.19133955289561697

In [ ]:
timestamp_list = []
for i in tqdm(range(len(accepted_ids))):
    timestamp_list.append(get_min_max_time(accepted_ids, i))
#all(timestamp_list[i] <= timestamp_list[i+1] for i in range(len(timestamp_list) - 1))

accepted_ids_w_st_time = sorted([(id, time) for id, time in zip(accepted_ids, timestamp_list)], key=lambda x: x[1][0])
accepted_ids_w_en_time = sorted([(id, time) for id, time in zip(accepted_ids, timestamp_list)], key=lambda x: x[1][1])

ids_length = len(timestamp_list)
train_num = int(ids_length * 0.8)

train_ids = [pair[0] for pair in accepted_ids_w_st_time[:train_num]]
test_ids = [pair[0] for pair in accepted_ids_w_en_time[train_num:]]

overlap = list(set(train_ids) & set(test_ids))
len(overlap)

print((len(test_ids)-len(overlap))/(len(timestamp_list)-len(overlap)))

test_ids = list(set(test_ids) - set(overlap))

train_df = df_application.loc[df_application['case:concept:name'].isin(train_ids)]
test_df = df_application.loc[df_application['case:concept:name'].isin(test_ids)]

train_df.to_csv('approved_train.csv')
test_df.to_csv('approved_test.csv')

#
#train_event_log = pm4py.convert_to_event_log(train_df)
#pm4py.write_xes(train_event_log, 'approved_train.xes')
#
#test_event_log = pm4py.convert_to_event_log(test_df)
#pm4py.write_xes(test_event_log, 'approved_test.xes')

# Part 1

In [58]:
def create_train_test(df_application, ids, outcome):

    timestamp_list = []
    for i in tqdm(range(len(ids))):
        timestamp_list.append(get_min_max_time(ids, i))
    #all(timestamp_list[i] <= timestamp_list[i+1] for i in range(len(timestamp_list) - 1))
    
    ids_w_st_time = sorted([(id, time) for id, time in zip(ids, timestamp_list)], key=lambda x: x[1][0])
    ids_w_et_time = sorted([(id, time) for id, time in zip(ids, timestamp_list)], key=lambda x: x[1][1])
    
    ids_length = len(timestamp_list)
    train_num = int(ids_length * 0.8)
    
    train_ids = [pair[0] for pair in ids_w_st_time[:train_num]]
    test_ids = [pair[0] for pair in ids_w_et_time[train_num:]]
    
    overlap = list(set(train_ids) & set(test_ids))
    
    print((len(test_ids)-len(overlap))/(len(timestamp_list)-len(overlap)))
    
    test_ids = list(set(test_ids) - set(overlap))
    
    train_df = df_application.loc[df_application['case:concept:name'].isin(train_ids)]
    test_df = df_application.loc[df_application['case:concept:name'].isin(test_ids)]
    
    train_df.to_csv(f'{outcome}_train.csv')
    test_df.to_csv(f'{outcome}_test.csv')

    return train_df, test_df
    
    

In [59]:
d_train_df, d_test_df = create_train_test(df_application, declined_ids, 'declined')

100%|██████████| 3752/3752 [03:54<00:00, 16.01it/s]


0.19175868569889576


In [60]:
c_train_df, c_test_df = create_train_test(df_application, cancelled_ids, 'cancelled')

100%|██████████| 10431/10431 [10:48<00:00, 16.09it/s]


0.19131614654002713


In [61]:
a_train_df, a_test_df = create_train_test(df_application, accepted_ids, 'approved')

100%|██████████| 17228/17228 [17:48<00:00, 16.12it/s]


0.19133955289561697


In [49]:
def save_xes(train_df, test_df, pre):

    train_event_log = pm4py.convert_to_event_log(train_df)
    pm4py.write_xes(train_event_log, f'{pre}_train.xes')

    test_event_log = pm4py.convert_to_event_log(test_df)
    pm4py.write_xes(test_event_log, f'{pre}_test.xes')

# Part 2 building dataset

In [ ]:
def aggregate_df(df):
    """ 
    Aggregate the df of current events in the case

    Output: 
        result -> could be a pandas series
    """

    # record the timestamp of the last activity
    result = df.iloc[0]

    return result


In [ ]:
def add_to_aggregate(result, df_row):
    """ 
    When a new event happens, add the event info to the current aggregated result.

    Input: 
         result: the current aggregated result
         df_row: pandas df row representing the new event
    Output:
        result: the new aggregated result
    """

    # record the timestamp of the last activity
    result = df_row.iloc[0]

    return result

In [34]:
def create_prefix_part2(df_application, app_ids, end_event, start_event='A_Accepted'):
    
    app_id_list = list(df_application['case:concept:name'].unique())

    # TODO:
    # create a return df
    return_df = pd.DataFrame()

    # extracting prefix for each application
    for app_id in app_id_list:
        
        events_app = df_application.loc[df_application['case:concept:name'] == app_id]
        events_app.reset_index(drop=True, inplace=True)

        # A_Accepted happens at most 1 time in each case
        # Otherwise will give error - only consider the first A_Accepted
        cur_id = starting_row_id = events_app.loc[events_app['concept:name'] == 'A_Accepted'].index[0]
        pre_events = events_app.iloc[:starting_row_id]
        # TODO: 
        # aggregate events_app from row 0 to starting_row_id
        result = aggregate_df(pre_events)
        
        ending_row_id = events_app.loc[events_app['concept:name'] == end_event].index[0]
        cur_id += 1
        
        while cur_id < ending_row_id:
            new_row = events_app.iloc[cur_id]
            # TODO: 
            # add new event row info to the aggregated result
            result = add_to_aggregate(pre_events)

            # Update the return_df -> add new row
            # target y: end_event

            cur_id += 1

        return return_df


# Part 3 building dataset


In [35]:
#build function to retrieve minimum time 
def get_min_time(ids, i):
    """define function to retrieve the minimum time"""
    return min(df_application.loc[df_application["case:concept:name"] == ids[i]]["time:timestamp"])

#get the minimum time for the accepted ids
min_timestamp_list = []
for i in tqdm(range(len(accepted_ids))):
    min_timestamp_list.append(get_min_time(accepted_ids, i))

#create a list with every id and the start time, and another list with every id and the end time
accepted_ids_begin = sorted([(id, time) for id, time in zip(accepted_ids, min_timestamp_list)], key=lambda x: x[1])
accepted_ids_end = sorted([(id, time) for id, time in zip(accepted_ids, timestamp_list)], key=lambda x: x[1])

#generate dataframe for begin and end times 
df_accepted_ids_time_begin = pd.DataFrame(accepted_ids_begin, columns = ["case:concept:name", "begin"])
df_accepted_ids_time_end = pd.DataFrame(accepted_ids_end, columns = ["case:concept:name", "end"])

#merge dataframes on case:concept:name
df_accepted_timestamps = df_accepted_ids_time_begin.merge(df_accepted_ids_time_end, on = "case:concept:name")

#keep relevant time formatting
df_accepted_timestamps["begin"] = df_accepted_timestamps["begin"].map(lambda x: str(x)[:19])
df_accepted_timestamps["end"] = df_accepted_timestamps["end"].map(lambda x: str(x)[:19])

#create function to calculate the difference in time from a dataframe with two columns containing dates and time
def calc_duration(end, begin):
    """calculate the difference in time using datetime.strptime"""
    return (datetime.strptime(end, "%Y-%m-%d %H:%M:%S") - datetime.strptime(begin, "%Y-%m-%d %H:%M:%S")).total_seconds()

#empty list to gather differences
duration = []

#retrieve the difference between begin and end of the trace and add to a list 
for i in range(0, len(df_accepted_timestamps)):
    duration.append(calc_duration(df_accepted_timestamps.iloc[i]["end"], df_accepted_timestamps.iloc[i]["begin"]))

#add the time difference to the df 
df_accepted_timestamps["duration"] = duration

#remove all cases with case time duration 0 
df_accepted_timestamps = df_accepted_timestamps.loc[df_accepted_timestamps["duration"] > 0]

#get indexes to filter outliers top and bottom 5%
outliers_index = list(range(0, round(0.05 * len(df_accepted_timestamps)))) + list(range(round(0.95 * len(df_accepted_timestamps)), len(df_accepted_timestamps)))

#sort values from small to big time difference and drop respective rows
df_accepted_timestamps = df_accepted_timestamps.sort_values(by= "duration", ignore_index = True).drop(labels = outliers_index, axis = "index")

In [ ]:
df_accepted_timestamps_begin = df_accepted_timestamps.sort_values(by = "begin").reset_index(drop = True)
df_accepted_timestamps_end = df_accepted_timestamps.sort_values(by = "end").reset_index(drop = True)

#test set range
begin_index_test = round(0.8 * len(df_accepted_timestamps_begin))
begin_time_test = df_accepted_timestamps_begin.iloc[begin_index_test]["begin"]
x = calc_duration(max(df_accepted_timestamps["end"]), begin_time_test)

#train set range
end_index_train = round(0.8 * len(df_accepted_timestamps_end))
end_time_train = df_accepted_timestamps_end.iloc[end_index_train]["end"]
y = calc_duration(end_time_train, min(df_accepted_timestamps["begin"]))
 
total_with_overlap = x + y
total_time = calc_duration(max(df_accepted_timestamps["end"]), min(df_accepted_timestamps["begin"]))
overlap_span = total_with_overlap - total_time

overlap_train = 0.8 * overlap_span
overlap_test = 0.2 * overlap_span

end_time_train_with_overlap = datetime.strptime(end_time_train, "%Y-%m-%d %H:%M:%S") 
date_index_train = datetime.strftime((end_time_train_datetime - timedelta(seconds = overlap_train)), "%Y-%m-%d %H:%M:%S")

begin_time_test_with_overlap = datetime.strptime(begin_time_test, "%Y-%m-%d %H:%M:%S") 
date_index_test = datetime.strftime((begin_time_test_datetime + timedelta(seconds = overlap_test)), "%Y-%m-%d %H:%M:%S")

df_train = df_accepted_timestamps_end.loc[df_accepted_timestamps_end["end"] < date_index_train]
df_test = df_accepted_timestamps_begin.loc[df_accepted_timestamps_begin["begin"] > date_index_test]

# Part 4 building dataset
---

#### First XOR - test: 

        A_Complete -> 

In [76]:
df_cancel_tr = pm4py.convert_to_dataframe(pm4py.read_xes('./../../Assignment_2/Data/cancelled_train.xes'))
df_cancel_te = pm4py.convert_to_dataframe(pm4py.read_xes('./../../Assignment_2/Data/cancelled_test.xes'))

parsing log, completed traces :: 100%|██████████| 2087/2087 [00:05<00:00, 371.85it/s]


In [77]:
df_cancel_tr.to_csv('./../../Assignment_2/Data/cancelled_train.csv')
df_cancel_te.to_csv('./../../Assignment_2/Data/cancelled_test.csv')

In [3]:
df_cancel_tr = pd.read_csv('./../../Assignment_2/Data/cancelled_train.csv')
df_cancel_te = pd.read_csv('./../../Assignment_2/Data/cancelled_test.csv')

In [93]:
all_columns = list(df_cancel_tr.columns)
all_columns

['Unnamed: 0',
 'Action',
 'org:resource',
 'concept:name',
 'EventOrigin',
 'EventID',
 'lifecycle:transition',
 'time:timestamp',
 'FirstWithdrawalAmount',
 'NumberOfTerms',
 'Accepted',
 'MonthlyCost',
 'Selected',
 'CreditScore',
 'OfferedAmount',
 'OfferID',
 'case:LoanGoal',
 'case:ApplicationType',
 'case:concept:name',
 'case:RequestedAmount']

In [129]:
case_attr = all_columns[all_columns.index('FirstWithdrawalAmount'):]

event_attr_cat = ['org:resource', 'concept:name', 'lifecycle:transition']
event_attr_num = ['time:timestamp']


In [139]:
result_df_columns = case_attr + ['time_to_current'] + ['last_time:timestamp']

resources = list(df_cancel_tr['org:resource'].unique())

df_cancel_tr['event_w_lifecycly'] = df_cancel_tr.apply(lambda row: row['concept:name'].replace(' ', '_') + '_' + row['lifecycle:transition'], axis=1)
events = list(df_cancel_tr['concept:name'].unique())
lifecycles = list(df_cancel_tr['lifecycle:transition'].unique())
events_w_lifecycle = list(df_cancel_tr['event_w_lifecycly'].unique())

In [140]:
result_df_columns.extend(events)
result_df_columns.extend(lifecycles)
result_df_columns.extend(events_w_lifecycle)

In [141]:
return_df = pd.DataFrame(columns=result_df_columns)

In [142]:
return_df
result_columns = list(return_df.columns)

In [126]:
app_id_list = list(df_cancel_tr['case:concept:name'].unique())
for app_id in tqdm(app_id_list):

    events_app = df_cancel_tr.loc[df_cancel_tr['case:concept:name'] == app_id]
    events_app.reset_index(drop=True, inplace=True)

    starting_rows = events_app.loc[events_app['concept:name'] == 'A_Complete']
    starting_row_ids = list(starting_rows.index)
    
    for starting_row_id in starting_row_ids:
        pre_events = events_app.iloc[:starting_row_id+1]
        
        # aggregate events_app from row 0 to starting_row_id
        result = aggregate_df(pre_events, result_columns)
        

100%|██████████| 8344/8344 [01:39<00:00, 84.21it/s]


In [ ]:
def aggregate_df(df, result_columns):
    """ 
    Aggregate the df of current events in the case

    Output: 
        result -> could be a pandas series
    """

    result = {}

    # Encode each case attribute as a feature (or one-hot encode categorical case attributes)
    for column in df.columns:
        pass

    # For each numerical event attribute, apply an aggregation function (e.g. average) over the sequence of values taken by this attribute in the prefix
    # sum up time as one feature?

    # For each categorical event attribute, encode each possible value of that attribute as a numerical feature. 

    return None

In [135]:
events_app = df_cancel_tr.loc[df_cancel_tr['case:concept:name'] == app_id_list[0]]

In [169]:
start_time = np.datetime64(events_app.iloc[0]['time:timestamp'])
end_time = np.datetime64(events_app.iloc[-1]['time:timestamp'])

duration = end_time - start_time
duration

/var/folders/nd/m2w3gccj0hx7zslrqlw6v9z80000gn/T/ipykernel_1142/2758137824.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  start_time = np.datetime64(events_app.iloc[0]['time:timestamp'])
/var/folders/nd/m2w3gccj0hx7zslrqlw6v9z80000gn/T/ipykernel_1142/2758137824.py:2: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  end_time = np.datetime64(events_app.iloc[-1]['time:timestamp'])


numpy.timedelta64(2743216489000,'us')

In [ ]:
result = {}

#['time_to_current'] + ['last_time:timestamp']

for column in events_app.columns:

    if 'case:' in column:
        result[column] = events_app.iloc[-1][column]
    elif 'timestamp' in column:
        result['last_time:timestamp'] = events_app.iloc[-1]['time:timestamp']
        result['time_to_current'] = events_app.iloc[-1]['time:timestamp'] - events_app.iloc[0]['time:timestamp']
    

    

In [ ]:
def create_dataset_part4_first_xor(df_application, current_event='O_Created', train=True, result_columns=None):
    """
    Encode original applications 
    """
    
    app_id_list = list(df_application['case:concept:name'].unique())

    # TODO: get all unique activities before target_event
    # TODO: get all unique resources before target_event
    # create a return df
    return_df = pd.DataFrame()
    # use abrove info to aggregate below

    for app_id in app_id_list:

        events_app = df_application.loc[df_application['case:concept:name'] == app_id]
        events_app.reset_index(drop=True, inplace=True)

        starting_row_id = events_app.loc[events_app['concept:name'] == current_event].index[0]
        pre_events = events_app.iloc[:starting_row_id]
        # TODO: 
        # aggregate events_app from row 0 to starting_row_id
        result = aggregate_df(pre_events)

        # aggregate rows of events and append the timestamp of the target_event

        # Encode each case attribute as a feature (or one-hot encode categorical case attributes)

        # For each numerical event attribute, apply an aggregation function (e.g. average) over the sequence of values taken by this attribute in the prefix
        # sum up time as one feature?

        # For each categorical event attribute, encode each possible value of that attribute as a numerical feature. 
        
        
        